In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import folium
from streamlit_folium import folium_static
import json
from pandas import json_normalize
import unicodedata
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
# Liens de téléchargement des données
# data20XX : allocataires APA puy de dome de 20XX
data2019_url = (
    "https://app.puy-de-dome.fr/open-data/public/files/app-open-data/Allocataires_APA_par_commune_(spatial_points)_1460_2020-09-03.csv")

data2020_url = (
    "https://app.puy-de-dome.fr/open-data/public/files/app-open-data/Allocataires_APA_par_commune_(spatial_points)_1492_2021-03-18.csv")

data2018_url = (
    "https://app.puy-de-dome.fr/open-data/public/files/app-open-data/Allocataires_APA_par_commune_(spatial_points)_1458_2020-09-03.csv")

data2017_url = (
    "https://app.puy-de-dome.fr/open-data/public/files/app-open-data/Allocataires_APA_par_commune_(spatial_points)_1456_2020-09-03.csv")

data2016_url = (
    "https://app.puy-de-dome.fr/open-data/public/files/app-open-data/Allocataires_APA_par_commune_(spatial_points)_1454_2020-09-03.csv")

data2015_url = (
    "https://app.puy-de-dome.fr/open-data/public/files/app-open-data/Allocataires_APA_par_commune_(spatial_points)_1452_2020-09-03.csv")

data2014_url = (
    "https://app.puy-de-dome.fr/open-data/public/files/app-open-data/Allocataires_APA_par_commune_(spatial_points)_1450_2020-09-03.csv")

# datageo : coordonnées des communes du puy de dome
datageocommune_url = (
    'https://github.com/gregoiredavid/france-geojson/raw/master/departements/63-puy-de-dome/communes-63-puy-de-dome.geojson')
geocommune=pd.read_json(datageocommune_url)

datageodepartement_url = (
'https://github.com/gregoiredavid/france-geojson/raw/master/departements/63-puy-de-dome/departement-63-puy-de-dome.geojson')
geodepartement=pd.read_json(datageodepartement_url)

df2020 = pd.read_csv(data2020_url, sep=';', encoding = "ISO-8859-1")
df2019 = pd.read_csv(data2019_url, sep=';', encoding = "ISO-8859-1")
df2018 = pd.read_csv(data2018_url, sep=';', encoding = "ISO-8859-1")
df2017 = pd.read_csv(data2017_url, sep=';', encoding = "ISO-8859-1")
df2016 = pd.read_csv(data2016_url, sep=';', encoding = "ISO-8859-1")
df2015 = pd.read_csv(data2015_url, sep=';', encoding = "ISO-8859-1")
df2014 = pd.read_csv(data2014_url, sep=';', encoding = "ISO-8859-1")

2021-10-08 15:00:57.708 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


In [3]:
# Notebook de nettoyage des données

col_suppr = ['DATE_DEBUT', 'DATE_FIN', 'ID_HISTORIQUE']
STR = ['CODE_COMMUNE', 'CODE_CANTON']
NBRE = ['TOTAL', 'TOTAL_FEM', 'TOTAL_HOM', 'TOTAL_M75A', 'TOTAL_7584A', 'TOTAL_P85A', 'TOTAL_DOM', 'TOTAL_ETA']

def nettoie(A):
    A.columns = A.loc[0]
    A.drop(0,axis=0,inplace=True)
    A.drop(col_suppr, axis=1, inplace=True)
    A.fillna(0, inplace=True)
    A[STR] = A[STR].astype('str')
    A[NBRE] = A[NBRE].astype('int')
    return A

nettoie(df2020)
nettoie(df2019)
nettoie(df2018)
nettoie(df2017)
nettoie(df2016)
nettoie(df2015)
nettoie(df2014)

def remove_accents(input_str):
    nkfd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nkfd_form if not unicodedata.combining(c)])

dfgeocommune = json_normalize(geocommune['features'])
dfgeocommune.drop('type', axis=1, inplace=True)
dfgeocommune.drop('geometry.type', axis=1, inplace=True)
dfgeocommune=dfgeocommune.rename(columns={"geometry.coordinates": "coordinates",
                      "properties.code": "CODE_COMMUNE",
                      "properties.nom": "LIBL_COMMUNE"})
dfgeocommune['LIBL_COMMUNE']=dfgeocommune['LIBL_COMMUNE'].str.upper()
long,lat=[],[]
for commune in range(len(dfgeocommune)):
    A,B, nbcom=0,0,len(dfgeocommune['coordinates'][commune][0])
    for point in range(nbcom):
        A+= (dfgeocommune['coordinates'][commune][0][point][0] if commune!=244 else dfgeocommune['coordinates'][commune][0][0][point][0])
        B+= (dfgeocommune['coordinates'][commune][0][point][1] if commune!=244 else dfgeocommune['coordinates'][commune][0][0][point][1])
    long.append(A/nbcom)
    lat.append(B/nbcom)

dfgeocommune['LONG']=long
dfgeocommune['LAT']=lat

for commune in range(len(dfgeocommune['LIBL_COMMUNE'])):
    dfgeocommune['LIBL_COMMUNE'][commune] = remove_accents(dfgeocommune['LIBL_COMMUNE'][commune])

    liste_coord=[]
for i in range(len(dfgeocommune)):
    point=[]
    for j in range(len(dfgeocommune['coordinates'][i][0])):
        truc=[]
        truc.append(dfgeocommune['coordinates'][i][0][j][1])
        truc.append(dfgeocommune['coordinates'][i][0][j][0])
        point.append(truc)
    liste_coord.append(point)
dfgeocommune['geo']=liste_coord

dfgeocommune.drop(['coordinates'], axis=1, inplace=True)

<ipython-input-3-9f12032ffd05>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfgeocommune['LIBL_COMMUNE'][commune] = remove_accents(dfgeocommune['LIBL_COMMUNE'][commune])


In [5]:
#Export des fichiers
dfgeocommune.to_csv('commune_geo.csv')

df2020.to_csv('APA_2020.csv')
df2019.to_csv('APA_2019.csv')
df2018.to_csv('APA_2018.csv')
df2017.to_csv('APA_2017.csv')
df2016.to_csv('APA_2016.csv')
df2015.to_csv('APA_2015.csv')
df2014.to_csv('APA_2014.csv')